In [1]:
from music21 import stream, interval, corpus, converter, instrument, metadata
from music21 import note, chord, environment, duration, clef, pitch, key
from music21.stream import Score, Part, Measure
import notebook
from datetime import date, timedelta
from collections import Counter
import random, math, re
import sys
import pandas as pd
import pathlib, random, copy
from builtins import isinstance
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
verbose = 1

In [2]:
#sys.path.append('C:\\Compile\\dwbzen\\dwbzen')
from dwbzen.music import MusicUtils, Instruments
from dwbzen.common import Geometry


In [3]:
import pandas as pd
notes_df = pd.read_csv('resources/music/bwv371_notes_df.csv')
counts_df = pd.read_csv('resources/music/bwv371_noteCounts_dpc_01.csv')
notes_df.columns

Index(['Unnamed: 0', 'note', 'part_number', 'part_name', 'name',
       'nameWithOctave', 'pitch', 'pitchClass', 'duration', 'type', 'ordinal',
       'quarterLength', 'quarterLengthNoTuplets'],
      dtype='object')

In [4]:
notes_df = notes_df[['part_name','part_number','name','pitch','pitchClass']]

In [5]:
notes_df['name'].value_counts()

D     135
A     120
E     117
G     115
C      97
B      85
F      57
F#     36
B-     19
C#     19
G#     11
E-      7
Name: name, dtype: int64

In [6]:
import pandas as pd
notes_df = pd.read_csv('resources/music/bwv371_notes_df.csv')
#notes_df = pd.read_csv("resources/music/random_score_notes_df.csv")
#notes_df = pd.read_csv('resources/music/bwv4xx_notes_df.csv')
#notes_df = pd.read_csv('resources/music/Visualization_Test_notes_df.csv')

## Two consecutive notes as lines drawn on a 24-sided polygon
with each side divided into n-segments (2 <= n <= 50).<br>
"from" notes are labeled across the top of each side: B, C, C#...<br>
The "to" notes are labeled across the bottom in the same order.<br>
Each 2-note sequence is realized as a line connecting "from" to "to" using the chosen color pallete.<br>

All the visualization modules take a Pandas DataFrame created by NoteCollector as input.<br>
Here is an example command line:<br>

--order 1  --source "\Compile\music21\music21\corpus\bach\bwv371.mxl" --format csv --name "bwv371" --parts "Soprano,Alto,Tenor,Bass"  --mode dpc

This creates a number of csv-formatted output files including a MarkovChain that can be used to generate music with the
MusicProducer class. The output file for visualization is 'bwv371_notes_df.csv' located in $PROJECT_HOME/resources/music/



In [7]:
#
# 
# 
#
from turtle import *
import pandas as pd

def main():
    
    sides_map_from = {"C":11, "C#":10, "D":9, "E-":8, "E":7, "F":6, "F#":5, "G":4, "G#":3, "A":2, "B-":1, "B":0}
    sides_map_to   = {"C":12, "C#":13, "D":14, "E-":15, "E":16, "F":17, "F#":18, "G":19, "G#":20, "A":21, "B-":22, "B":23}
    
    #part_names = ['Violin I','Violin II','Viola', 'Cello']
    part_names = ['Soprano','Alto','Tenor', 'Bass']

    colors1 = ['#FF0000','#fa6705','#fad905','#adfb04','#44f906','#04c12f', \
              '#06f9ad', '#05d0fa','#0663f9','#2306f9','#9106f9','#f906f0']

    colors2 = ['red','blue','orange','indigo','OrangeRed4','green',\
              'DarkCyan', 'hot pink','LightGreen','MediumPurple','yellow','blue violet']
    #colors2.reverse()

    color_palette = [colors1, colors2]

    fillcolors = ['blue','orange','indigo','OrangeRed4','green','DarkCyan', 
                  'hot pink','LightGreen','MediumPurple','yellow','blue violet','red']

    palette = 1
    colors = color_palette[palette]
    draw_polygon = True
    verbose = 1

    #
    # the sides of the 24-gon map to pitches
    # each side divides into n segments and so has a len of n+1, the first being a polygon vertex point
    #
    nsides = 24
    radius = 400
    nsegments = 50
    points = Geometry.polygon_points(nsides, radius)
    poly_seg_points = Geometry.polygon_segment_points(nsides, radius, nsegments)

    counters = [Counter() for i in range(nsides)]
    for i in range(nsides):
        counters[i] = [0 for x in range(nsegments)]

    #
    # draw the polygon outline and vertext points
    #
    setup(width=900, height=900)
    color('black')
    dot(4)   # center of the plot
    color('blue')   # polygon points color
    penup()
    speed(0)
    tracer(3,0)
    width(2)
    if draw_polygon:
        for p in points:
            goto(p)
            pendown()
            dot(6)
    color('red')   # segment points color
    penup()
    hideturtle()

    only_these_from_notes = []
    only_these_to_notes = []
    
    bgpic(picname='/Compile/dwbzen/resources/music/notes24.gif')

    #
    # connect the 'from' notes to the 'to' notes
    #
    width(1)
    for partname in part_names:
        part_df = notes_df[notes_df['part_name']==partname]
        nnotes = len(part_df)
        if verbose > 0:
            print(f"{partname}: {nnotes} notes" )
        for i in range(0, nnotes-1):
            # i is the index of the 'from' note, i+1 the 'to' note
            from_note = part_df.iloc[i]['name']
            to_note = part_df.iloc[i+1]['name']
            #
            # optional filtering to, from notes
            #
            if len(only_these_from_notes) > 0 and from_note not in only_these_from_notes:
                continue
            if len(only_these_to_notes) > 0 and to_note not in only_these_to_notes:
                continue

            from_side = sides_map_from[from_note]
            to_side = sides_map_to[to_note]
            if verbose > 1:
                print(f"{from_note} {from_side}\t{to_note} {to_side}")
            counter_from = counters[from_side]
            counter_to = counters[to_side]
            seg_points_from = poly_seg_points[from_side]
            seg_points_to = poly_seg_points[to_side]
            ind_from_point = 0
            ind_to_point = 0
            if 0 in counter_from:
                ind_from_point = counter_from.index(0)
                counter_from[ind_from_point] = 1
            else:    # reset the counters
                counters[from_side] = [0]*nsegments

            if 0 in counter_to:
                ind_to_point = counter_to.index(0)
                counter_to[ind_to_point] = 1
            else:    # reset the counters
                counters[to_side] = [0]*nsegments

            from_point = seg_points_from[ind_from_point] 
            #from_point = seg_points_from[nsegments - ind_from_point - 1]    # reverse order
            to_point = seg_points_to[ind_to_point]
            penup()
            color(colors[from_side])
            goto(from_point)
            pendown()
            goto(to_point)
    print("Done")

if __name__ == '__main__':
    main()
    mainloop()


Soprano: 132 notes
Alto: 225 notes
Tenor: 241 notes
Bass: 220 notes
Done


In [46]:
points = Geometry.polygon_points(24, 400)
poly_seg_points = Geometry.polygon_segment_points(24, 400, 4)

## Two consecutive notes as lines drawn on a 12-sided polygon
The thickness of the line is determined by the frequency of the note pair.<br>
Note that to- and from- a given pair are represented by a single line so the direction is lost in this rendering.<br>
Choose the vertex mapping (i.e. the order of the pitches around the circle) as chromatic or circle of fifths


In [8]:
import pandas as pd
#notes_df = pd.read_csv('resources/music/bwv371_notes_df.csv')
notes_df = pd.read_csv('resources/music/op18.1_1_notes_df.csv')
# notes_df = pd.read_csv('resources/music/bwv437_notes_df.csv')
# notes_df = pd.read_csv('resources/music/bwv4xx_notes_df.csv')

In [10]:
#
# Visualize a sequence of two consecutive notes as lines drawn on a 12-sided polygon
# TODO improve performance
#

from turtle import *
import pandas as pd

def main():
    chromatic_vertex_map = {"C":11, "C#":10, "D":9, "E-":8, "E":7, "F":6, "F#":5, "G":4, "G#":3, "A":2, "B-":1, "B":0}
    circle_of_fifths_vertex_map = {"C":0, "C#":7, "D":2, "E-":9, "E":4, "F":11, "F#":6, "G":1, "G#":8, "A":3, "B-":10, "B":5}
    vertex_maps = [chromatic_vertex_map, circle_of_fifths_vertex_map]

    # choose the vertex map to use
    vertex_map = vertex_maps[1]

    #
    # if no parts are specified, assume all parts
    #
    part_names = None   # ['Violin I','Violin II','Viola', 'Cello'] 
    #part_names = ['Soprano','Alto','Tenor', 'Bass']

    colors1 = ['#FF0000','#fa6705','#fad905','#adfb04','#44f906','#04c12f', \
              '#06f9ad', '#05d0fa','#0663f9','#2306f9','#9106f9','#f906f0']
    colors1 += colors1

    colors2 = ['red','blue','orange','indigo','OrangeRed4','green',\
              'DarkCyan', 'hot pink','LightGreen','MediumPurple','DarkGreen','blue violet']
    colors2 += colors2

    color_palette = [colors1, colors2]


    palette = 1
    colors = color_palette[palette]
    verbose = 1
    monochrome_color = 'blue'    # set to None to use the color_palette

    #
    # the verticies of the 12-gon map to pitches in a chromatic or circle of fifths progression 
    #
    nsides = 12
    radius = 400
    nsegments = 2
    points = Geometry.polygon_points(nsides, radius)
    poly_seg_points = Geometry.polygon_segment_points(nsides, radius, nsegments)

    #
    # draw the polygon outline and vertext points
    #
    setup(width=900, height=900)
    color('black')
    dot(4)   # center of the plot
    penup()
    speed(0)
    tracer(4,0)
    width(2)
    i = 0
    for p in points:
        color(colors[i])
        goto(p)
        #pendown()
        dot(6)
        i = i+ 1
    penup()
    hideturtle()
    #
    # keep a count of from, to pairs
    #
    pair_counts = dict()
    bgpic(picname='/Compile/dwbzen/resources/music/fifths.gif')
    
    #
    # connect note to next note verticies
    #
    width(1)
    if part_names is None:
        part_names = notes_df['part_name'].unique().tolist()
    for partname in part_names:
        part_df = notes_df[notes_df['part_name']==partname]
        nnotes = len(part_df)
        if verbose > 0:
            print(f"{partname}: {nnotes} notes" )
        for i in range(0, nnotes-1):
            # i is the index of the 'from' note, i+1 the 'to' note
            from_note = part_df.iloc[i]['name']
            to_note = part_df.iloc[i+1]['name']

            pair_key = from_note+to_note
            if pair_key in pair_counts:
                pair_counts[pair_key] = pair_counts[pair_key] + 1
            else:
                pair_counts[pair_key] = 1

        pc_list = list(pair_counts.values())
        rng = max(pc_list) - min(pc_list)
        pc_series = pd.Series(data=pair_counts)
        pc_scaled = dict()
        for k in pair_counts.keys():
            pc_scaled[k] = round(pair_counts[k]/rng * 9)  # a scaled value in range(10) (0 to 9)

        for i in range(0, nnotes-1):
            # i is the index of the 'from' note, i+1 the 'to' note
            from_note = part_df.iloc[i]['name']
            to_note = part_df.iloc[i+1]['name']
            index_from = vertex_map[from_note]
            point_from = points[index_from]
            index_to = vertex_map[to_note]
            point_to = points[index_to]
            if verbose > 1:
                print(f"from: {from_note} to: {to_note}")

            goto(point_from)
            if monochrome_color is None:
                color(colors[index_from])
            else:
                color(monochrome_color)
            width(1 + pc_scaled[from_note+to_note])
            pendown()
            goto(point_to)
            penup()
    print("Done!")

        
if __name__ == '__main__':
    main()
    mainloop()


Violin I: 1350 notes
Violin II: 885 notes
Done!


## Pairs - two consecutive notes as lines drawn on a 12-sided polygon
Each vertext is a pair representing the to- and from- the note and so the direction is maintained.<br>
The thickness of the line is determined by the frequency of the note pair.<br>
Choose the vertex mapping (i.e. the order of the pitches around the circle) as chromatic or circle of fifths.<br>
Choose the pallete or set monochrome_color.

In [16]:
import pandas as pd
notes_df = pd.read_csv('resources/music/bwv371_notes_df.csv')
#notes_df = pd.read_csv('resources/music/op18.1_1_notes_df.csv')
#notes_df = pd.read_csv('resources/music/Visualization_Test_notes_df.csv')

In [19]:
#
# Visualize a sequence of 
# The thickness of the line is determined by the frequency of the note pair
# the vertex maps determine the order of pitches around the circle
#
from turtle import *
import pandas as pd

def main():

    chromatic_vertex_map = {"C":0, "C#":1, "D":2, "E-":3, "E":4, "F":5, "F#":6, "G":7, "G#":8, "A":9, "B-":10, "B":11}
    circle_of_fifths_vertex_map = {"C":0, "G":1, "D":2, "A":3, "E":4, "B":5, "F#":6, "C#":7, "G#":8,"E-":9, "B-":10, "F":11}
                                  
    vertex_maps = [chromatic_vertex_map, circle_of_fifths_vertex_map]

    # choose the vertex map to use
    vertex_map = vertex_maps[0]

    #
    # if no parts are specified, assume all parts
    #
    part_names = None  # ['Violin I','Violin II','Viola', 'Cello'] # ['Soprano','Alto','Tenor', 'Bass']

    colors1 = ['#FF0000','#fa6705','#fad905','#adfb04','#44f906','#04c12f', \
              '#06f9ad', '#05d0fa','#0663f9','#2306f9','#9106f9','#f906f0']

    colors2 = ['red','blue','orange','indigo','OrangeRed4','green',\
              'DarkCyan', 'hot pink','LightGreen','MediumPurple','yellow','blue violet']
    #colors2.reverse()

    color_palette = [colors1, colors2]

    fillcolors = ['blue','orange','indigo','OrangeRed4','green','DarkCyan', 
                  'hot pink','LightGreen','MediumPurple','yellow','blue violet','red']

    palette = 1
    colors = color_palette[palette]
    draw_polygon = False
    verbose = 1
    monochrome_color = None    # set to None to use the color_palette
    
    max_notes = None    # set to a number to limit number of notes/part
    max_width = None    # set to a number to make all connection lines the same width - useful for debugging
    width_scale = 10     # scales the width from 1 to width_scale

    #
    # the sides of the 12-gon map to pitches
    # each side divides into 2 segments and so has a len of 3, the first being a polygon vertex point
    #
    nsides = 12
    radius = 400
    nsegments = 5   # must be an odd number. Increase to pull to,from points closer together
    middle_point_index = nsegments//2

    points = Geometry.polygon_points(nsides, radius)
    all_poly_seg_points = Geometry.polygon_segment_points(nsides, radius, nsegments)
    #
    # reduce to 2 points per segment, omiting the polygon vertex point
    # so only the middle 2 points - for from- and to- pitches respectively
    #
    poly_seg_points = {}
    for i in range(len(all_poly_seg_points)):
        pts = all_poly_seg_points[i]
        poly_seg_points[i] = [pts[middle_point_index], pts[middle_point_index+1]]


    counters = [[] for i in range(nsides)]
    for i in range(nsides):
        counters[i] = [0 for x in range(nsegments)]

    #
    # draw the polygon outline and vertext points
    #
    setup(width=900, height=900)
    bgpic(picname='/Compile/dwbzen/resources/music/notes-fifths-pairs.gif')
    color('black')
    dot(4)   # center of the plot
    color('blue')   # polygon points color
    penup()
    speed(0)
    tracer(5,0)
    width(1)

    if draw_polygon:
        for p in points:
            goto(p)
            pendown()
            dot(4)
            #penup()

    penup()
    hideturtle()
    #
    # keep a count of from, to pairs
    #
    pair_counts = dict()
    #
    # draw the 2 segment points and connecting line
    # for each point in poly_seg_points
    #
    for i in range(len(poly_seg_points)):
        points = poly_seg_points[i]
        goto(points[0])    # this will be the from- pitch - red
        pendown()
        color('red')
        dot(8)
        color('blue')
        goto(points[1])    # this will be the to-pitch - green
        color('green')
        dot(8)
        penup()

    #
    # connect note to next note verticies
    #
    width(1)
    if part_names is None:
        part_names = notes_df['part_name'].unique().tolist()
    for partname in part_names:
        part_df = notes_df[notes_df['part_name']==partname]
        #
        # limit number of notes
        #
        if max_notes is not None:
            nnotes = max_notes
        else:
            nnotes = len(part_df)

        if verbose > 0:
            print(f"{partname}: {nnotes} notes" )
        #
        # pair_counts is the number of occurrences of the from,to combination
        # and determines the line thickness
        #
        for i in range(0, nnotes-1):
            # i is the index of the 'from' note, i+1 the 'to' note
            from_note = part_df.iloc[i]['name']
            to_note = part_df.iloc[i+1]['name']

            pair_key = from_note+to_note
            if pair_key in pair_counts:
                pair_counts[pair_key] = pair_counts[pair_key] + 1
            else:
                pair_counts[pair_key] = 1

        pc_list = list(pair_counts.values())
        rng = max(pc_list) - min(pc_list)
        pc_series = pd.Series(data=pair_counts)
        pc_scaled = dict()
        
        # scale the pair_count value from 1 to width_scale
        for k in pair_counts.keys():
            pc_scaled[k] = round(pair_counts[k]/rng * width_scale)  


        for i in range(0, nnotes-1):
            # i is the index of the 'from' note, i+1 the 'to' note
            from_note = part_df.iloc[i]['name']
            to_note = part_df.iloc[i+1]['name']

            index_from = vertex_map[from_note]
            index_to = vertex_map[to_note]

            point_from = poly_seg_points[index_from][0]
            point_to = poly_seg_points[index_to][1]

            if verbose > 1:
                print(f"from: {from_note} ({index_from})  at {point_from}\tto: {to_note} ({index_to}) at {point_to}")

            goto(point_from)
            if monochrome_color is None:
                color(colors[index_from])
            else:
                color(monochrome_color)
            if max_width is None:
                width(1 + pc_scaled[from_note+to_note])
            else:
                width(max_width)
            pendown()
            goto(point_to)
            penup()
    print("Done!")
    return "Done!"

if __name__ == '__main__':
    main()
    mainloop()


Soprano: 132 notes
Alto: 225 notes
Tenor: 241 notes
Bass: 220 notes
Done!


In [10]:
counters = [[] for i in range(5)]
for i in range(5):
    cnt = Counter()
    for s in range(10):
        cnt[s] = 0
    counters[i] = cnt
counters
    

[Counter({0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}),
 Counter({0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}),
 Counter({0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}),
 Counter({0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}),
 Counter({0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0})]